In [17]:
from spacerocks import SpaceRock, Units
import numpy as np

from astropy import units as u
from astropy.time import Time
import matplotlib.pyplot as plt
%matplotlib inline

units = Units()
units.timescale = 'tdb'
#units.current()

#rock = SpaceRock(a=4.327825657409375E+02, 
#                 e=9.185882992879753E-01, 
#                 inc=5.413121974184833E+01, 
#                 node=1.350508167296842E+02, 
#                 arg=3.482436748740575E+02, 
#                 M=3.584598907033508E+02, 
#                 epoch=24589?82.500000000, 
#                 H0=4.4,
#                 name='2015 BP519',
#                 frame='heliocentric',
#                 units=units)

rock = SpaceRock(a=2.765985763166805E+00, 
                 e=7.821081922804583E-02, 
                 inc=1.058793079235974E+01, 
                 node=8.027114319577151E+01, 
                 arg=7.373012101495385E+01, 
                 M=2.108992101403974E+02, 
                 epoch=2459225.500000000, 
                 H0=3.53,
                 G=0.12,
                 name='Ceres',
                 frame='heliocentric',
                 units=units)

#rock = SpaceRock(x=2.899956856761939E+00, 
#                 y=4.830845049759105E-02, 
#                 z=-5.340760077627554E-01, 
#                 vx=-4.466056534856986E-04, 
#                 vy=9.633109832631537E-03, 
#                 vz=3.857239030382092E-04, 
#                 epoch=2459225.500000000, 
#                 H0=3.53,
#                 G=0.12,
#                 name='Ceres',
#                 frame='barycentric',
#                 units=units)


units = Units()
units.timescale = 'utc'

p = rock.propagate([2459225.5, 2459226.5], model=2, units=units)

In [18]:
%precision 16
o = rock.observe(obscode=500)
op = p.observe(obscode=500)

In [19]:
o.ra.hms, o.dec.dms, o.delta, o.mag

(hms_tuple(h=array([23.]), m=array([18.]), s=array([41.541927457506915])),
 dms_tuple(d=array([-14.]), m=array([-14.]), s=array([-14.258486358416036])),
 <Quantity [3.4125073607222514] AU>,
 <Quantity [9.42012466072214]>)

In [20]:
op.ra.hms, op.dec.dms, op.delta, op.mag

(hms_tuple(h=array([23., 23.]), m=array([18., 19.]), s=array([41.59835643528538, 52.27028611716861])),
 dms_tuple(d=array([-14., -14.]), m=array([-14.,  -4.]), s=array([-13.791327334221961, -29.506334694537486])),
 <Quantity [3.4125162361559487, 3.423550889821151 ] AU>,
 <Quantity [9.420126481255286, 9.42232009584144 ]>)

## Earth

2459225.500000000 = A.D. 2021-Jan-11 00:00:00.0000 TDB 

X =-3.541466815795880E-01 

Y = 9.259818046222517E-01 

Z = 6.579327077401820E-05

VX=-1.638956931455718E-02 

VY=-6.147606423658539E-03 

VZ= 1.230618652396049E-06


In [ ]:
from skyfield.api import Topos, Loader
# Load in planets for ephemeride calculation.
load = Loader('./Skyfield-Data', expire=True, verbose=True)
ts = load.timescale(builtin=False)
planets = load('de441.bsp')

In [ ]:
t = ts.tdb(jd=rock.epoch.tdb.jd)
earth = planets['earth']

In [ ]:
t1 = ts.tai(jd=rock.epoch.tdb.jd + 32.184/86400)
t2 = ts.tdb(jd=rock.epoch.tdb.jd)

In [ ]:
ts.tdb_jd(t1.whole, t1.tdb_fraction - 0.3)

In [ ]:
help(ts.tdb_jd)

In [ ]:
(JDTDB.jd - JDUTC.jd) * 86400

In [ ]:
t1.tdb - t1.ut1 - t1.dut1/86400

In [ ]:
t1.dut1

In [ ]:
e = earth.at(t)

In [ ]:
from skyfield.framelib import ecliptic_J2000_frame

In [ ]:
x, y, z = e.position.au

In [ ]:
from astropy.coordinates import Angle
epsilon = Angle(84381.448, u.arcsec)

In [ ]:
print(x, y * np.cos(epsilon) + z * np.sin(epsilon), y * np.sin(epsilon) - z * np.cos(epsilon))

In [ ]:
%precision 16
print(e.ecliptic_xyz().au)
print(e.ecliptic_velocity().au_per_d)

In [ ]:
x, y, z = e.position.au

In [ ]:
np.sqrt(x**2 + y**2 + z**2)

In [ ]:
from spacerocks import SpaceRock, Units
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

units = Units()
units.current()

N = 3
rocks = SpaceRock(a=np.random.uniform(30, 100, N), 
                  e=np.random.rand(N) * 0.5, 
                  inc=np.random.uniform(0, 90, N), 
                  node=np.random.uniform(0, 360, N), 
                  arg=np.random.uniform(0, 360, N), 
                  M=np.random.uniform(0, 360, N), 
                  epoch=np.repeat(2459305.5, N), 
                  H0=np.random.uniform(4, 10, N),
                  #delta_H = np.random.uniform(0.1, 0.5, N),
                  #rotation_period = np.random.uniform(0.2, 1, N),
                  #phi0 = np.random.rand(N) * 360,
                  name=[str(x) for x in range(N)],
                  frame='barycentric',
                  units=units)

p = rocks.propagate(epochs=np.linspace(2459305.5, 2459315.5, 100), model=2)
o = p.observe(obscode='W84')

colors = [float(hash(s) % 256) / 256 for s in o.name]

fig, ax = plt.subplots(2, 1, figsize=(18, 12), sharex=True)
ax[0].scatter((o.epoch.jd - o.epoch.jd[0]) / 365.25, o.mag, s=1, c=colors, cmap='turbo')
ax[0].tick_params(labelsize=16)
ax[0].set_ylabel('mag', fontsize=24, labelpad=30)

ax[1].scatter((o.epoch.jd - o.epoch.jd[0]) / 365.25, o.H, s=1, c=colors, cmap='turbo')
ax[1].tick_params(labelsize=16)
ax[1].set_xlabel(r'$\Delta t$ (years)', fontsize=24, labelpad=30)
ax[1].set_ylabel('H', fontsize=24, labelpad=30, rotation=0)

fig.tight_layout()

In [ ]:
rocks.to_helio()

In [ ]:
p.H0

In [ ]:
N = 3
rocks = SpaceRock(a=np.random.uniform(30, 100, N), 
                  e=np.random.rand(N) * 0.5, 
                  inc=np.random.uniform(0, 90, N), 
                  node=np.random.uniform(0, 360, N), 
                  arg=np.random.uniform(0, 360, N), 
                  M=np.random.uniform(0, 360, N), 
                  epoch=np.repeat(2459305.5, N), 
                  H0=np.random.uniform(4, 10, N),
                  delta_H=np.random.uniform(0.1, 0.5, N),
                  rotation_period=np.random.uniform(0.2, 1, N),
                  phi0 = np.random.rand(N) * 360,
                  name=np.array([str(x) for x in range(N)]),
                  frame='barycentric',
                  units=units)

In [ ]:
rocks.t_peri
rocks.true_anomaly
rocks.mean_longitude
rocks.true_longitude
rocks.p
rocks.b
rocks.q
rocks.Q
rocks.n
rocks.rrdot
rocks.r
rocks.TisserandJ
rocks.frame
rocks.mu

In [ ]:
rocks.__dict__

In [ ]:
np.atleast_1d(np.random.uniform(0.0, 0.6, 10))

In [ ]:
rocks[rocks.a.au > 50].frame

In [ ]:
rocks.__dict__

In [ ]:
np.ones(100) * rocks.epoch.jd

In [ ]:
np.atleast_1d(10)

In [ ]:
len(rocks.epoch.jd)

In [ ]:
from spacerocks import SpaceRock, Units

In [ ]:
spacerocks.__version__

In [ ]:
dict = {'name':rocks.name, 
        'epoch':rocks.epoch, 
        'a':rocks.a.au, 
        'e':rocks.e, 
        'inc':rocks.inc.deg, 
        'node':rocks.node.deg, 
        'arg':rocks.arg.deg,
        'M':rocks.M.deg,
        'H0':rocks.H0,
        'delta_H':rocks.delta_H,
        'rotation_period':rocks.rotation_period,
        'phi0':rocks.phi0.deg}
df = pd.DataFrame.from_dict(dict)

In [ ]:
df

In [ ]:
import spacerocks

In [ ]:
spacerocks.__version__

In [ ]:
from spacerocks import SpaceRock, Units

In [ ]:
units.angle = 'rad'

In [ ]:
units.current()

In [ ]:
import pandas as pd
from astropy import units as u
dict = {'name':o.name, 
        'epoch':o.epoch, 
        'ra':o.ra.deg, 
        'dec':o.dec.deg, 
        'ra_rate':o.ra_rate.to(u.deg/u.day), 
        'dec_rate':o.dec_rate.to(u.deg/u.day), 
        'mag':o.mag,
        'H':o.H}
df = pd.DataFrame.from_dict(dict)

In [ ]:
df

In [ ]:
o.dec_rate

In [ ]:
N = 10
rock = SpaceRock(a=np.random.uniform(30, 100, N), 
                 e=np.random.rand(N), 
                 inc=np.random.uniform(0, 90, N), 
                 node=np.random.uniform(0, 360, N), 
                 arg=np.random.uniform(0, 360, N), 
                 M=np.random.uniform(0, 360, N), 
                 epoch=np.repeat(2459305.5, N), 
                 H0=np.random.uniform(4, 10, N),
                 delta_H = np.random.uniform(0.1, 0.5, N), # amplitude
                 rotation_period = np.random.uniform(0.2, 1, N), # period in days
                 phi0 = np.random.rand(N) * 360, # initial phase in radians
                 frame='barycentric',
                 units=units)

p = rock.propagate(epochs=np.linspace(2459305.5, 2499955.5, 10000), model=2)
o = p.observe(obscode='W84')

colors = [float(hash(s) % 256) / 256 for s in o.name]

fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((o.epoch.jd - o.epoch.jd[0]) / 365.25, o.mag, s=1, c=colors, cmap='turbo')
ax.tick_params(labelsize=16)
ax.set_xlabel(r'$\Delta t$ (years)', fontsize=24, labelpad=30)
ax.set_ylabel('mag', fontsize=24, labelpad=30)

fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((o.epoch.jd - o.epoch.jd[0]) / 365.25, o.H, s=1, c=colors, cmap='turbo')
ax.tick_params(labelsize=16)
ax.set_xlabel(r'$\Delta t$ (years)', fontsize=24, labelpad=30)
ax.set_ylabel('H', fontsize=24, labelpad=30, rotation=0)

In [ ]:
N = 1
rock = SpaceRock(a=30, 
                 e=0, 
                 inc=0, 
                 node=0, 
                 arg=0, 
                 M=0, 
                 H=5,
                 delta_H=np.array([0.5]),
                 phi0=np.array([0]),
                 rotation_period=np.array([0.25]),
                 epoch=np.repeat(2459305.5, N), 
                 frame='barycentric', units=units)

In [ ]:
pr = rock.propagate(epochs=np.linspace(2459305.5, 2459355.5, 50), model=2)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))

true = []
estimated = []
elong = []

N = 10000000
rock = SpaceRock(a=np.random.uniform(30, 500, N), 
                 e=np.random.rand(N), 
                 inc=np.random.uniform(0, 90, N), 
                 node=np.random.uniform(0, 360, N), 
                 arg=np.random.uniform(0, 360, N), 
                 M=np.random.uniform(0, 360, N), 
                 epoch=np.repeat(2459305.5, N), 
                 frame='barycentric', units=units)


t0 = ts.tdb(jd=rock.epoch.tdb.jd)
t1 = ts.tdb(jd=rock.epoch.tdb.jd + 4/24)
earth = planets['earth']

# Only used for the topocentric calculation.
#if self.__class__.obscode != 500:
#    earth += Topos(latitude_degrees=self.__class__.obslat,
#                   longitude_degrees=self.__class__.obslon,
#                   elevation_m=self.__class__.obselev) # topocentric calculation

e0 = earth.at(t0)
e1 = earth.at(t1)
x0, y0, z0 = e0.ecliptic_xyz().au
x1, y1, z1 = e1.ecliptic_xyz().au

d_earth = sqrt((x1 - x0)**2 + (y1 - y0)**2 + (z1 - z0)**2)

obs = Observe(rock)

rate = sqrt(obs.ra_rate.to(u.arcsec/u.hour)**2 * cos(obs.dec)**2 + obs.dec_rate.to(u.arcsec/u.hour)**2)

estimated.append(d_earth / tan(rate * 4 *u.h))
true.append(sqrt(obs.xT.au**2 + obs.yT.au**2 + obs.zT.au**2))
    
true = array(true)
estimated = array(estimated)

error = abs(true - estimated) / true
    


In [ ]:
fig, ax = plt.subplots(figsize=(18, 18))

sc = ax.scatter(obs.elong.deg, error, alpha=0.01, s=1, color='black')
    

ax.tick_params(labelsize=14)
ax.set_xlabel('elongation', fontsize=24, labelpad=20)
ax.set_ylabel('Distance Relative Error', fontsize=24, labelpad=20)
ax.set_xlim([0, 180]);
ax.set_ylim([0.001, 1])
ax.set_yscale('log');